In [1]:
import cobrakbase

cobrakbase 0.2.2


In [3]:
kbase = cobrakbase.KBaseAPI('UGOG6KLAWTCYI2ASYECYHNIIFTEXGA2J')

In [134]:
%run ../lib/mightymorphingmodels/morph.py
%run ../lib/mightymorphingmodels/service.py
%run ../lib/mightymorphingmodels/objects.py


In [140]:
%run ../lib/mightymorphingmodels/GrowthConditions.py
growth_condition = CobraCondition()
growth_condition.evaluate({'morph': morph, 'model': translated_model})

None
48522


AttributeError: 'NoneType' object has no attribute 'objective'

In [127]:
#genome = kbase.get_object('Methanocaldococcus_jannaschii_DSM_2661', ws_id)
#protcomp = kbase.get_object('maripaludis_jannaschii', ws_id)
probanno = kbase.get_object('Methanosphaera_stadtmanae_DSM_3091.rxnprobs', ws)
#media = kbase.get_object('2016_01_19_h2co2only_media', ws_id)
#src_model = kbase.get_object('MaripaludisModelRevised', ws_id)
translated_model = kbase.get_object('translated_MaripaludisModelRevisedCleanIds', ws)
#recon_model = kbase.get_object('recon_Methanosphaera_stadtmanae_DSM_3091', ws)
super_modelspc = kbase.get_object('super_modelspc', ws)

In [128]:
#src_model = FBAModel('MaripaludisModelRevised', ws_id, data=src_model)
translated_model = FBAModel('translated_MaripaludisModelRevised', ws, data=translated_model)
#recon_model = FBAModel('recon_Methanocaldococcus_jannaschii_DSM_2661', ws_id, data=recon_model)
#genome = Genome('Methanocaldococcus_jannaschii_DSM_2661', ws_id, data=genome)
#protcomp = ProteomeComparison('maripaludis_jannaschii', ws_id, data=protcomp)
probanno = ReactionProbabilities('Methanosphaera_stadtmanae_DSM_3091.rxnprobs', ws_id, data=probanno)
#media = Media('2016_01_19_h2co2only_media', ws_id, data=media)
super_modelspc = FBAModel('super_modelspc', ws_id, data=super_modelspc)

In [64]:
translated_model

Type: <class '__main__.FBAModel'>, Stored Identity: ('translated_MaripaludisModelRevised', 'bking:narrative_1568786629037')

In [42]:
src_model = None
translated_model = None
recon_model = None
genome = None
protcomp = None
probanno = None
media = None
super_modelspc = None

In [84]:
probanno

Type: <class '__main__.ReactionProbabilities'>, Stored Identity: ('Methanosphaera_stadtmanae_DSM_3091.rxnprobs', '48522')

In [48]:
service = Service(None, None, None)

In [83]:
morph.probanno

In [125]:
type(morph.probanno)

__main__.ReactionProbabilities

In [123]:
ReactionProbabilities

__main__.ReactionProbabilities

In [132]:
isinstance(morph.probanno, ReactionProbabilities)
#assert isinstance(morph.probanno, ReactionProbabilities), str(type(probanno))

True

In [133]:
morph = Morph(service=service,
              src_model=src_model,
              media=media,
              probanno=probanno,
              protcomp=protcomp,
              genome=genome,
              ws_id=ws_id)
morph.probanno=probanno
morph.check_rep()


service <__main__.Service object at 0x7f8d30abea90>
src_model None
media None
probanno Type: <class '__main__.ReactionProbabilities'>, Stored Identity: ('Methanosphaera_stadtmanae_DSM_3091.rxnprobs', '48522')
protcomp None
genome None
ws_id 48522


AssertionError: <class '__main__.ReactionProbabilities'>

In [103]:
getattr(morph, 'probanno')

In [118]:
print(morph.probanno)

Type: <class '__main__.ReactionProbabilities'>, Stored Identity: ('Methanosphaera_stadtmanae_DSM_3091.rxnprobs', '48522')


In [8]:
# Get a Workspace Service Client
ws_id = '48522'
ws_data = kbase.ws_client.get_workspace_info({'id': ws_id})
ws = ws_data[1]
ws

'bking:narrative_1568786629037'

In [9]:
# Fetch new / old Morph Model
def get_model_from_ws(model_name, workspace_id=ws_id):
    return ws.get_objects2({'objects': [{'wsid': workspace_id, 'name': model_name}]})['data'][0]

new_model = kbase.get_object('maripaludis_to_stadtmanae_morph_9_17_19', ws)
old_model = kbase.get_object('MarpaludisStadtmanaeMorphOld', ws)

In [12]:
# Simple Comparison of Model Contents based on Reaction Ids. Some previous models have identifiers with extra un-used 
# compartment tags, that no longer cooperate with KBase methods for reaction comparisons as expected. To simplify comparison,
# remove: E.g. rxn_12345-c0-_c0 -> rxn_12345_c0.

def get_rxn_id(r):
    # perform id clean
    return r['id'].replace('-c0-','')

new_rxns = dict([(get_rxn_id(r) , r) for r in new_model['modelreactions']])
old_rxns = dict([(get_rxn_id(r), r) for r in old_model['modelreactions']])

new_only = set(new_rxns.keys()) - set(old_rxns.keys())
old_only = set(old_rxns.keys()) - set(new_rxns.keys())
both = set(new_rxns.keys()).intersection(set(old_rxns.keys()))

print('Number of Reactions found only in the new morph', len(new_only))
print('Number of Reactions found only in the old morph', len(old_only))
print('Number of Reactions found in both', len(both))

Number of Reactions found only in the new morph 10
Number of Reactions found only in the old morph 80
Number of Reactions found in both 589


In [19]:
a = cobrakbase.core.KBaseFBAModel(new_model)
#b = cobrakbase.core.KBaseFBAModel(old_model) #dup stuff

In [16]:

len(set(new_rxns))

599

In [20]:
len(set(new_rxns.keys()))

599

In [21]:
# Method for Counting Gapfills. Takes in set of reaction ids to get subset from, and a dict<str, reaction> where str is string id ('rxn_12345_c0') 
# and reaction is a dict defined by model['data']['modelreactions']. All ids in target set need to be in rxn_dict.
def gapfilled_subset(target_set, rxn_dict):
    gapfills = set()
    for r in target_set:
        is_gapfill = True
        for p in rxn_dict[r]['modelReactionProteins']:
            for s in p['modelReactionProteinSubunits']:
                if len(s['feature_refs']) > 0:
                    is_gapfill = False
                    break
        if is_gapfill:
            gapfills.add(r)
    return gapfills
        
print('In the New-Only subset, {0} of {1} are likely gapfilled reactions'.format(len(gapfilled_subset(new_only, new_rxns)), len(new_only)))
print('In the Old-Only subset, {0} of {1} are likely gapfilled reactions'.format(len(gapfilled_subset(old_only, old_rxns)), len(old_only)))


In the New-Only subset, 2 of 10 are likely gapfilled reactions
In the Old-Only subset, 79 of 80 are likely gapfilled reactions


In [22]:
# Method for Counting Gapfills. Takes in set of reaction ids to get subset from, and a dict<str, reaction> where str is string id ('rxn_12345_c0') 
# and reaction is a dict defined by model['data']['modelreactions']. All ids in target set need to be in rxn_dict.
def gapfilled_subset(target_set, rxn_dict):
    gapfills = set()
    for r in target_set:
        is_gapfill = True
        for p in rxn_dict[r]['modelReactionProteins']:
            for s in p['modelReactionProteinSubunits']:
                if len(s['feature_refs']) > 0:
                    is_gapfill = False
                    break
        if is_gapfill:
            gapfills.add(r)
    return gapfills
        
print('In the New-Only subset, {0} of {1} are likely gapfilled reactions'.format(len(gapfilled_subset(new_only, new_rxns)), len(new_only)))
print('In the Old-Only subset, {0} of {1} are likely gapfilled reactions'.format(len(gapfilled_subset(old_only, old_rxns)), len(old_only)))


In the New-Only subset, 2 of 10 are likely gapfilled reactions
In the Old-Only subset, 79 of 80 are likely gapfilled reactions


In [23]:
# Detect duplicate reactions in a model
def detect_duplicate_rxns(model_name):
    m = get_model_from_ws(model_name)
    dupes = set()
    rxns = set()
    for r in m['data']['modelreactions']:
        rxn_id = r['id']
        if rxn_id in rxns:
            dupes.add(rxn_id)
        else:
            rxns.add(rxn_id)
    return dupes
detect_duplicate_rxns('maripaludis_to_stadtmanae_morph_9_17_19')

AttributeError: 'str' object has no attribute 'get_objects2'

In [27]:

# We should enumerate these to see if there are any interesting patterns metabolically
for r in old_rxns:
    if r in old_only:
        print(r, '->', old_rxns[r]['name'])


rxn05667_c0 -> Urea transport via facilitate diffusion c0
rxn00062_c0 -> ATP phosphohydrolase protein-secreting c0
rxn01208_c0 -> 2-Oxo-4-methyl-3-carboxypentanoate decarboxylation c0
EX-cpd15302_c0 -> EX glycogenn-1 c0
rxn10904_c0 -> Acetate transport
rxn01117_c0 -> D-Arabinose-5-phosphate ketol-isomerase_c0
rxn01360_c0 -> (S)-Dihydroorotate:oxygen oxidoreductase_c0
rxn04385_c0 -> L-threonine-O-3-phosphate carboxy-lyase_c0
rxn00148_c0 -> ATP:pyruvate O2-phosphotransferase_c0
rxn02312_c0 -> S-Adenosyl-L-methionine:8-amino-7-oxononanoate aminotransferase_c0
rxn00469_c0 -> N2-Acetyl-L-ornithine amidohydrolase_c0
rxn00321_c0 -> L-Lysine 2,3-aminomutase_c0
rxn05064_c0 -> carbamate hydro-lyase_c0
rxn05527_c0 -> cytosine transport in via proton symport_c0
rxn00283_c0 -> L-Alanine racemase_c0
rxn00351_c0 -> gamma-L-Glutamyl-L-cysteine:glycine ligase (ADP-forming)_c0
rxn05651_c0 -> sulfate transport in via proton symport_c0
rxn10596_c0 -> (R)-(homo)3citrate dehydratase_c0
rxn05197_c0 -> Uracil